In [1]:
! pip install mysql-connector-python
! pip install pandas

import pandas as pd
import mysql.connector
from tabulate import tabulate
from datetime import datetime

# Data Cleaning and Preparation 

In [2]:


# Load the data from CSV files
customers_df = pd.read_csv(r'F:\NITHYA ONLINE DATA SCIENCE\PROJECT-2 DATA SPARK\DATA SET\Customers.csv', encoding='latin1')
exchange_rates_df = pd.read_csv(r'F:\NITHYA ONLINE DATA SCIENCE\PROJECT-2 DATA SPARK\DATA SET\Exchange_Rates.csv', encoding='latin1')
products_df = pd.read_csv(r'F:\NITHYA ONLINE DATA SCIENCE\PROJECT-2 DATA SPARK\DATA SET\Products.csv', encoding='latin1')
sales_df = pd.read_csv(r'F:\NITHYA ONLINE DATA SCIENCE\PROJECT-2 DATA SPARK\DATA SET\Sales.csv', encoding='latin1')
stores_df = pd.read_csv(r'F:\NITHYA ONLINE DATA SCIENCE\PROJECT-2 DATA SPARK\DATA SET\Stores.csv', encoding='latin1')

In [3]:
customers_df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,7/3/1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,9/27/1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,5/26/1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,9/17/1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,11/19/1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duï¿½kovï¿½,Houston,TX,Texas,77017,United States,North America,3/25/1936
15262,2099618,Male,Justin Solï¿½rzano,Mclean,VA,Virginia,22101,United States,North America,2/16/1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,11/9/1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,10/12/1937


In [4]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerKey  15266 non-null  int64 
 1   Gender       15266 non-null  object
 2   Name         15266 non-null  object
 3   City         15266 non-null  object
 4   State Code   15256 non-null  object
 5   State        15266 non-null  object
 6   Zip Code     15266 non-null  object
 7   Country      15266 non-null  object
 8   Continent    15266 non-null  object
 9   Birthday     15266 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.2+ MB


In [5]:
customers_df.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [6]:

# Create a new column 'State_Info' with the desired format
customers_df['Customer State_Details'] = customers_df.apply(lambda row: f"{row['State']} ({row['State Code']})-{row['Zip Code']}", axis=1)

# Drop the original 'State', 'State Code', and 'Zip Code' columns if you don't need them anymore
customers_df.drop(columns=['State', 'State Code', 'Zip Code'], inplace=True)

customers_df=customers_df.rename(columns={'Country':'Customer Country',
                                          'Name':'Customer Name',
                                          'Gender':'Customer Gender',
                                          'City':'Customer City',
                                          'Continent':'Customer Continent',
                                          'Birthday':'Customer Birthday',
                                        })

In [7]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15266 entries, 0 to 15265
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerKey             15266 non-null  int64 
 1   Customer Gender         15266 non-null  object
 2   Customer Name           15266 non-null  object
 3   Customer City           15266 non-null  object
 4   Customer Country        15266 non-null  object
 5   Customer Continent      15266 non-null  object
 6   Customer Birthday       15266 non-null  object
 7   Customer State_Details  15266 non-null  object
dtypes: int64(1), object(7)
memory usage: 954.3+ KB


In [8]:
sales_df

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,NaN,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [9]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order Number   62884 non-null  int64 
 1   Line Item      62884 non-null  int64 
 2   Order Date     62884 non-null  object
 3   Delivery Date  13165 non-null  object
 4   CustomerKey    62884 non-null  int64 
 5   StoreKey       62884 non-null  int64 
 6   ProductKey     62884 non-null  int64 
 7   Quantity       62884 non-null  int64 
 8   Currency Code  62884 non-null  object
dtypes: int64(6), object(3)
memory usage: 4.3+ MB


In [10]:
sales_df.drop(['Delivery Date'],axis=1,inplace=True)
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order Number   62884 non-null  int64 
 1   Line Item      62884 non-null  int64 
 2   Order Date     62884 non-null  object
 3   CustomerKey    62884 non-null  int64 
 4   StoreKey       62884 non-null  int64 
 5   ProductKey     62884 non-null  int64 
 6   Quantity       62884 non-null  int64 
 7   Currency Code  62884 non-null  object
dtypes: int64(6), object(2)
memory usage: 3.8+ MB


In [11]:
products_df

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,$6.62,$12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,$6.62,$12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,$7.40,$14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,$11.00,$21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,$11.00,$21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,$43.07,$129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,$1.71,$3.35,505,Cell phones Accessories,5,Cell phones


In [12]:
products_df['Unit Cost USD'] = products_df['Unit Cost USD'].str.replace('$', '')
products_df['Unit Price USD'] = products_df['Unit Price USD'].str.replace('$', '')

In [13]:
products_df

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones


In [14]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ProductKey      2517 non-null   int64 
 1   Product Name    2517 non-null   object
 2   Brand           2517 non-null   object
 3   Color           2517 non-null   object
 4   Unit Cost USD   2517 non-null   object
 5   Unit Price USD  2517 non-null   object
 6   SubcategoryKey  2517 non-null   int64 
 7   Subcategory     2517 non-null   object
 8   CategoryKey     2517 non-null   int64 
 9   Category        2517 non-null   object
dtypes: int64(3), object(7)
memory usage: 196.8+ KB


In [15]:
stores_df

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [16]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreKey       67 non-null     int64  
 1   Country        67 non-null     object 
 2   State          67 non-null     object 
 3   Square Meters  66 non-null     float64
 4   Open Date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB


In [17]:
# Set 'Square Meters' to 0 where 'StoreKey' is 0
stores_df.loc[stores_df['StoreKey'] == 0, 'Square Meters'] = 0

stores_df=stores_df.rename(columns={'Country':'Store Country',
                                    'State':'Store State',
                                    'Square Meters':'Stores Square Meter',
                                    'Open Date':'Store Open Date'
                                    }
                        )


In [18]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   StoreKey             67 non-null     int64  
 1   Store Country        67 non-null     object 
 2   Store State          67 non-null     object 
 3   Stores Square Meter  67 non-null     float64
 4   Store Open Date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB


In [19]:
exchange_rates_df

,Date,Currency,Exchange
0,1/1/2015,USD,1.0000
1,1/1/2015,CAD,1.1583
2,1/1/2015,AUD,1.2214
3,1/1/2015,EUR,0.8237
4,1/1/2015,GBP,0.6415
...,...,...,...
11210,2/20/2021,USD,1.0000
11211,2/20/2021,CAD,1.2610
11212,2/20/2021,AUD,1.2723
11213,2/20/2021,EUR,0.8238


In [20]:
exchange_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      11215 non-null  object 
 1   Currency  11215 non-null  object 
 2   Exchange  11215 non-null  float64
dtypes: float64(1), object(2)
memory usage: 263.0+ KB


In [21]:
exchange_rates_df=exchange_rates_df.rename(columns={'Date':'Exchange Date',
                                                    'Currency':'Currency Code',
                                                    'Exchange':'Exchange Rate'
                                                    })

In [22]:
exchange_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11215 entries, 0 to 11214
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Exchange Date  11215 non-null  object 
 1   Currency Code  11215 non-null  object 
 2   Exchange Rate  11215 non-null  float64
dtypes: float64(1), object(2)
memory usage: 263.0+ KB


In [23]:

# Merge Sales with Customers on CustomerKey
sales_customers_df = pd.merge(sales_df, customers_df, how='left', on='CustomerKey')

# Merge the result with Products on ProductKey
sales_customers_products_df = pd.merge(sales_customers_df, products_df, how='left', on='ProductKey')

# Merge the result with Stores on StoreKey
sales_customers_products_stores_df = pd.merge(sales_customers_products_df, stores_df, how='left', on='StoreKey')

# Merge the result with Exchange Rates on Date and Currency Code
final_df = pd.merge(sales_customers_products_stores_df, exchange_rates_df, how='left', left_on=['Order Date', 'Currency Code'], right_on=['Exchange Date', 'Currency Code'])


# Display the final DataFrame
print(final_df.head())


   Order Number  Line Item Order Date  CustomerKey  StoreKey  ProductKey  \
0        366000          1   1/1/2016       265598        10        1304   
1        366001          1   1/1/2016      1269051         0        1048   
2        366001          2   1/1/2016      1269051         0        2007   
3        366002          1   1/1/2016       266019         0        1106   
4        366002          2   1/1/2016       266019         0         373   

   Quantity Currency Code Customer Gender   Customer Name  ... SubcategoryKey  \
0         1           CAD            Male    Tyler Vaught  ...            406   
1         2           USD            Male  Frank Upchurch  ...            402   
2         1           USD            Male  Frank Upchurch  ...            803   
3         7           CAD          Female      Joan Roche  ...            402   
4         1           CAD          Female      Joan Roche  ...            301   

                        Subcategory CategoryKey         

In [24]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Order Number            62884 non-null  int64  
 1   Line Item               62884 non-null  int64  
 2   Order Date              62884 non-null  object 
 3   CustomerKey             62884 non-null  int64  
 4   StoreKey                62884 non-null  int64  
 5   ProductKey              62884 non-null  int64  
 6   Quantity                62884 non-null  int64  
 7   Currency Code           62884 non-null  object 
 8   Customer Gender         62884 non-null  object 
 9   Customer Name           62884 non-null  object 
 10  Customer City           62884 non-null  object 
 11  Customer Country        62884 non-null  object 
 12  Customer Continent      62884 non-null  object 
 13  Customer Birthday       62884 non-null  object 
 14  Customer State_Details  62884 non-null

In [25]:
final_df.isnull().sum()

Order Number              0
Line Item                 0
Order Date                0
CustomerKey               0
StoreKey                  0
ProductKey                0
Quantity                  0
Currency Code             0
Customer Gender           0
Customer Name             0
Customer City             0
Customer Country          0
Customer Continent        0
Customer Birthday         0
Customer State_Details    0
Product Name              0
Brand                     0
Color                     0
Unit Cost USD             0
Unit Price USD            0
SubcategoryKey            0
Subcategory               0
CategoryKey               0
Category                  0
Store Country             0
Store State               0
Stores Square Meter       0
Store Open Date           0
Exchange Date             0
Exchange Rate             0
dtype: int64

In [34]:
final_df['Order Date'] = pd.to_datetime(final_df['Order Date'], errors='coerce').dt.strftime('%Y-%m-%d')
final_df['Customer Birthday'] = pd.to_datetime(final_df['Customer Birthday'], errors='coerce').dt.strftime('%Y-%m-%d')
final_df['Store Open Date'] = pd.to_datetime(final_df['Store Open Date'], errors='coerce').dt.strftime('%Y-%m-%d')
final_df['Exchange Date'] = pd.to_datetime(final_df['Exchange Date'], errors='coerce').dt.strftime('%Y-%m-%d')
final_df['Unit Price USD'] = final_df['Unit Price USD'].replace('[\$,]', '', regex=True).astype(float)
final_df['Unit Cost USD'] = final_df['Unit Cost USD'].replace('[\$,]', '', regex=True).astype(float)

In [40]:
# Check for any NaT values in the date columns
print(final_df[final_df['Customer Birthday'].isna()])
print(final_df[final_df['Order Date'].isna()])
print(final_df[final_df['Store Open Date'].isna()])
print(final_df[final_df['Exchange Date'].isna()])


Empty DataFrame
Columns: [Order Number, Line Item, Order Date, CustomerKey, StoreKey, ProductKey, Quantity, Currency Code, Customer Gender, Customer Name, Customer City, Customer Country, Customer Continent, Customer Birthday, Customer State_Details, Product Name, Brand, Color, Unit Cost USD, Unit Price USD, SubcategoryKey, Subcategory, CategoryKey, Category, Store Country, Store State, Stores Square Meter, Store Open Date, Exchange Date, Exchange Rate]
Index: []

[0 rows x 30 columns]
Empty DataFrame
Columns: [Order Number, Line Item, Order Date, CustomerKey, StoreKey, ProductKey, Quantity, Currency Code, Customer Gender, Customer Name, Customer City, Customer Country, Customer Continent, Customer Birthday, Customer State_Details, Product Name, Brand, Color, Unit Cost USD, Unit Price USD, SubcategoryKey, Subcategory, CategoryKey, Category, Store Country, Store State, Stores Square Meter, Store Open Date, Exchange Date, Exchange Rate]
Index: []

[0 rows x 30 columns]
Empty DataFrame
Co

In [27]:
all_data=final_df.to_csv(r'F:\NITHYA ONLINE DATA SCIENCE\PROJECT-2 DATA SPARK\DATA SET\final_data.csv', index=False)


# Connect to MySQL server

In [41]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="nithya",
    database="DataSpark"  
)
print("Database connection established.")

mycursor = mydb.cursor(buffered=True)
mydb.commit()

Database connection established.


In [44]:
# Commit any previous transactions
mydb.commit()

# Create the table if it doesn't exist
create_table_query = ''' CREATE TABLE IF NOT EXISTS `Global Electronics` (
    `Order Number` INT,
    `Line Item` INT,
    `Order Date` DATE,
    `CustomerKey` INT,
    `StoreKey` INT,
    `ProductKey` INT,
    `Quantity` INT,
    `Currency Code` VARCHAR(255),
    `Customer Gender` VARCHAR(255),
    `Customer Name` VARCHAR(255),
    `Customer City` VARCHAR(255),
    `Customer Country` VARCHAR(255),
    `Customer Continent` VARCHAR(255),
    `Customer Birthday` DATE,
    `Customer State_Details` VARCHAR(255),
    `Product Name` VARCHAR(255),
    `Brand` VARCHAR(255),
    `Color` VARCHAR(255),
    `Unit Cost USD` FLOAT,
    `Unit Price USD` FLOAT,
    `SubcategoryKey` INT,
    `Subcategory` VARCHAR(255),
    `CategoryKey` INT,
    `Category` VARCHAR(255),
    `Store Country` VARCHAR(255),
    `Store State` VARCHAR(255),
    `Stores Square Meter` FLOAT,
    `Store Open Date` DATE,
    `Exchange Date` DATE,
    `Exchange Rate` FLOAT
)
'''

# Execute the query to create the table
mycursor.execute(create_table_query)
mydb.commit()

print("Table created successfully.")


Table created successfully.


In [46]:
# Convert 'Customer Birthday' to datetime and format it
final_df['Customer Birthday'] = pd.to_datetime(final_df['Customer Birthday'], errors='coerce').dt.strftime('%Y-%m-%d')

# Prepare data for insertion
data_to_insert = final_df.to_records(index=False).tolist()

# SQL Insert query
insert_query = '''
INSERT INTO `Global Electronics` (
    `Order Number`,
    `Line Item`,
    `Order Date`,
    `CustomerKey`,
    `StoreKey`,
    `ProductKey`,
    `Quantity`,
    `Currency Code`,
    `Customer Gender`,
    `Customer Name`,
    `Customer City`,
    `Customer Country`,
    `Customer Continent`,
    `Customer Birthday`,
    `Customer State_Details`,
    `Product Name`,
    `Brand`,
    `Color`,
    `Unit Cost USD`,
    `Unit Price USD`,
    `SubcategoryKey`,
    `Subcategory`,
    `CategoryKey`,
    `Category`,
    `Store Country`,
    `Store State`,
    `Stores Square Meter`,
    `Store Open Date`,
    `Exchange Date`,
    `Exchange Rate`
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Execute the query with the prepared data
mycursor.executemany(insert_query, data_to_insert)
mydb.commit()

# Close the database connection
mycursor.close()
mydb.close()

print("Data inserted successfully.")


Data inserted successfully.
